In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import IPython

def display_prediction(text, predictions):
    class ObjectToDisplay(object):
        def _repr_html_(self):
            cmap = matplotlib.cm.get_cmap('Reds')
            html_chars = []
            for c, p in zip(text, predictions):
                if c == '\n':
                    html_chars.append('<br>')
                else:
                    r, g, b, a = cmap(p)
                    r, g, b = int(256*r), int(256*g), int(256*b)
                    html_chars.append('<span style="background-color:rgb(%s, %s, %s); color:black;">%s</span>' % (r, g, b, c))
            tot_html = "".join(html_chars)
            return tot_html
    return ObjectToDisplay()

text = 'polska krowa \n spalinowa'
targets = np.random.random(len(text))
display_prediction(text, targets)

In [3]:
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, TimeDistributed
from keras.datasets import imdb
# from keras.models import Input

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)


In [4]:
chars = '\n !"#$%&\'()*+,-./0123456789:;<=>?@[\\]^_`abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ{|}~'
charset = set(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


In [2]:
sklearn_path = '../data/sklearn.py'
austen_path = '../data/austen.txt'
merged_path = '../data/merged.txt'
labels_path = '../data/target.txt'
features_path = '../data/X'
target_path = '../data/y'

In [6]:
import joblib
y = joblib.load(target_path)

y, len(y)

(array([0, 0, 0, ..., 1, 1, 1]), 10260217)

In [5]:
import json

merged_path = '/home/nadbor/workspace/neural_playground/data/merged.txt'
target_path = '/home/nadbor/workspace/neural_playground/data/target.txt'

with open(merged_path, 'rb') as merged_file:
    merged = merged_file.read()
    
with open(target_path, 'rb') as target_file:
    target = json.load(target_file)

In [6]:
encoded = encode_text(merged)

In [7]:
n_train = 40000
n_test = n_train
n = n_train + n_test
maxlen = 90
n_chars = len(charset)
assert n_chars == 96
X_train = np.array(encoded[:n_train]).reshape((1, n_train, n_chars))
y_train = np.array(target[:n_train]).reshape((1, n_train, 1))

In [8]:
X_train.shape

(1, 40000, 96)

In [12]:
rnn_size = 4
epochs = 10
batch_size = 1

model = Sequential()
model.add(LSTM(rnn_size, return_sequences=True, input_shape=(n_train, n_chars)))
model.add(LSTM(rnn_size, return_sequences=True, input_shape=(n_train, n_chars), go_backwards=True))

model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(units=1, activation='sigmoid')))
model.compile(optimizer='adam', loss='mse', metrics=['accuracy', 'binary_crossentropy'])

model.summary()
model.fit(X_train, y_train, epochs=epochs)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 40000, 4)          1616      
_________________________________________________________________
lstm_4 (LSTM)                (None, 40000, 4)          144       
_________________________________________________________________
dropout_2 (Dropout)          (None, 40000, 4)          0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 40000, 1)          5         
Total params: 1,765.0
Trainable params: 1,765.0
Non-trainable params: 0.0
_________________________________________________________________
Epoch 1/10
1/1 [==============================] - 25s - loss: 0.2502 - acc: 0.4822 - binary_crossentropy: 0.6935
Epoch 2/10
1/1 [==============================] - 25s - loss: 0.2501 - acc: 0.4929 - binary_crossentropy: 0.6933
Epoch 3/10
1/1 [========================

In [13]:
preds = model.predict(X_train).ravel()

In [15]:
a = 13000
b = a+1000
display_prediction(merged[a:b], (preds[a:b] - 0.5) * 40 + 0.5)

In [12]:
from keras.models import load_model
mo = load_model('models/zero/austen_0_epochs_9.h5py')
mo.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1000000, 64)       41216     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000000, 64)       0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 1000000, 64)       33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000000, 64)       0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 1000000, 64)       33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000000, 64)       0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1000000, 1)        65        
Total para